In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras import losses, activations, models
from keras.layers import Dense, InputLayer,BatchNormalization, Dropout, Conv2D, Flatten, Activation

In [2]:
# ImageNet classification dataset for Resnet18 and ResNet50
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar100.load_data() #Size is (, 32,32,3)

In [3]:
# Normalize the input image so that each pixel value is between 0 to 1.
x_train = x_train / 255.0
x_test = x_test / 255.0

In [14]:
from tensorflow.keras.applications.resnet50 import ResNet50
def get_model():       
       model_Resnet50 = ResNet50(include_top=False,
                        weights='imagenet',
                        input_shape=(32, 32, 3),
                        pooling=None)
       return model_Resnet50


In [5]:
model_given = get_model()
model_given.summary()

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 38, 38, 3)    0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 16, 16, 64)   9472        ['conv1_pad[0][0]']              
                                                                                                  
 conv1_bn (BatchNormalization)  (None, 16, 16, 64)   256         ['conv1_conv[0][0]']             
                                                                                           

In [2]:
# to plot the model
!pip install netron
import netron
# model_given.save('model_given.h5')
netron.start('model_given.h5',8088)

Serving 'model_given.h5' at http://localhost:8088



[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: C:\Users\Nithya Raj\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


('localhost', 8088)

In [6]:
# Function to obtain the output from each layer of the model (To be used for the repeat task type)
output_size = {}
def keras_model_size(model):
    prefix_name = model.name + "_"
    for layer in model.layers:
        if isinstance(layer, (tf.keras.Model,models.Sequential)):
            keras_model_size(layer)
        else:
          out_shape = layer.output_shape
        output_size[prefix_name + layer.name] = out_shape
       
          
    return output_size


In [7]:
keras_model_size(model_given)

{'resnet50_input_1': [(None, 32, 32, 3)],
 'resnet50_conv1_pad': (None, 38, 38, 3),
 'resnet50_conv1_conv': (None, 16, 16, 64),
 'resnet50_conv1_bn': (None, 16, 16, 64),
 'resnet50_conv1_relu': (None, 16, 16, 64),
 'resnet50_pool1_pad': (None, 18, 18, 64),
 'resnet50_pool1_pool': (None, 8, 8, 64),
 'resnet50_conv2_block1_1_conv': (None, 8, 8, 64),
 'resnet50_conv2_block1_1_bn': (None, 8, 8, 64),
 'resnet50_conv2_block1_1_relu': (None, 8, 8, 64),
 'resnet50_conv2_block1_2_conv': (None, 8, 8, 64),
 'resnet50_conv2_block1_2_bn': (None, 8, 8, 64),
 'resnet50_conv2_block1_2_relu': (None, 8, 8, 64),
 'resnet50_conv2_block1_0_conv': (None, 8, 8, 256),
 'resnet50_conv2_block1_3_conv': (None, 8, 8, 256),
 'resnet50_conv2_block1_0_bn': (None, 8, 8, 256),
 'resnet50_conv2_block1_3_bn': (None, 8, 8, 256),
 'resnet50_conv2_block1_add': (None, 8, 8, 256),
 'resnet50_conv2_block1_out': (None, 8, 8, 256),
 'resnet50_conv2_block2_1_conv': (None, 8, 8, 64),
 'resnet50_conv2_block2_1_bn': (None, 8, 8, 64

In [8]:
# Obtains the output in bits, the memory requirements of each layer

def keras_model_output(model):
    output_dict = {}
    default_dtype = tf.keras.backend.floatx()
    prefix_name = model.name + "_"
    for layer in model.layers:
        if isinstance(layer, (tf.keras.Model,models.Sequential)):
            keras_model_output(layer)
        else:
            no_of_neurons = tf.as_dtype(layer.dtype or default_dtype).size # dtype is float 32 and the size here is 4 bytes(32 bits)
            out_shape = layer.output_shape
            if isinstance(out_shape, list):
                for s in out_shape[0]:
                    if s is None:
                        continue
                    no_of_neurons *= s # Output obtained here is in 4 Bytes 
            else:
                for s in out_shape:
                    if s is None:
                        continue
                    no_of_neurons *= s # Output obtained here is in 4 Bytes 
            no_of_neurons *= 8 # Output obtained in bits
        output_dict[prefix_name + layer.name] = no_of_neurons
       
          
    return output_dict

In [9]:
keras_model_output(model_given)

{'resnet50_input_1': 98304,
 'resnet50_conv1_pad': 138624,
 'resnet50_conv1_conv': 524288,
 'resnet50_conv1_bn': 524288,
 'resnet50_conv1_relu': 524288,
 'resnet50_pool1_pad': 663552,
 'resnet50_pool1_pool': 131072,
 'resnet50_conv2_block1_1_conv': 131072,
 'resnet50_conv2_block1_1_bn': 131072,
 'resnet50_conv2_block1_1_relu': 131072,
 'resnet50_conv2_block1_2_conv': 131072,
 'resnet50_conv2_block1_2_bn': 131072,
 'resnet50_conv2_block1_2_relu': 131072,
 'resnet50_conv2_block1_0_conv': 524288,
 'resnet50_conv2_block1_3_conv': 524288,
 'resnet50_conv2_block1_0_bn': 524288,
 'resnet50_conv2_block1_3_bn': 524288,
 'resnet50_conv2_block1_add': 524288,
 'resnet50_conv2_block1_out': 524288,
 'resnet50_conv2_block2_1_conv': 131072,
 'resnet50_conv2_block2_1_bn': 131072,
 'resnet50_conv2_block2_1_relu': 131072,
 'resnet50_conv2_block2_2_conv': 131072,
 'resnet50_conv2_block2_2_bn': 131072,
 'resnet50_conv2_block2_2_relu': 131072,
 'resnet50_conv2_block2_3_conv': 524288,
 'resnet50_conv2_block2

In [10]:
# EFFECTIVE DATA COMMUNICATION PROBABILITY

# Function to print the effective data communicated 
def data_summary(model,x_test,y_test):

  # Creating dictionary for storing the percent of data communicated eliminating the sparse data
  effective_probability = {}
  
  # Function to add communicated data to the metrics
  def add_metrics(m2):
    "Annotates recursiverly a (non-compiled) model with metrics that calculate probability of non-zero activity"
    prefix_name = m2.name + "_"

    for ll in m2.layers:
        if is_hier_layer(ll):
            add_metrics(ll)
        else:
            my_metric = tf.reduce_sum( tf.math.count_nonzero(ll.output, dtype=tf.int32) / tf.size( ll.output)  )
            name=prefix_name + ll.name
            m2.add_metric(my_metric, name=name)
        
    return

  # Function to check if a layer is model or not
  def is_hier_layer(layer):
      "Finds if layer is actually a model instead of a single layer"
      return type(layer) in [models.Sequential, keras.Model]

  # Clone the model. Note that it is not compiled!
  m2 = keras.models.clone_model(model)
  add_metrics(m2)
  
  # Compiling the cloned model and take the inference
  m2.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

  m2.evaluate(x_test[0:100], y_test[0:100]) 
  
  # print('%40s | %20s' % ('Metrics and Layer Name', 'Effective data communicated'))
  # print("*" * 150)
  for l in m2.metrics:
    # print('%40s | %5.4f' % (l.name, l.result().numpy()))
    if l.name!="loss" and l.name!="accuracy" :
      effective_probability[l.name] = l.result().numpy()      

  return effective_probability


In [11]:
data_summary(model_given, x_train, y_train)

4/4 [==============================] - 14s 109ms/step - loss: 10.1292 - accuracy: 0.0000e+00 - resnet50_input_1: 0.9591 - resnet50_conv1_pad: 0.6801 - resnet50_conv1_conv: 0.9900 - resnet50_conv1_bn: 0.9900 - resnet50_conv1_relu: 0.4800 - resnet50_pool1_pad: 0.3792 - resnet50_pool1_pool: 0.6477 - resnet50_conv2_block1_1_conv: 0.9980 - resnet50_conv2_block1_1_bn: 0.9980 - resnet50_conv2_block1_1_relu: 0.4412 - resnet50_conv2_block1_2_conv: 1.0000 - resnet50_conv2_block1_2_bn: 1.0000 - resnet50_conv2_block1_2_relu: 0.4906 - resnet50_conv2_block1_0_conv: 0.9980 - resnet50_conv2_block1_3_conv: 1.0000 - resnet50_conv2_block1_0_bn: 0.9980 - resnet50_conv2_block1_3_bn: 1.0000 - resnet50_conv2_block1_add: 1.0000 - resnet50_conv2_block1_out: 0.4507 - resnet50_conv2_block2_1_conv: 1.0000 - resnet50_conv2_block2_1_bn: 1.0000 - resnet50_conv2_block2_1_relu: 0.4670 - resnet50_conv2_block2_2_conv: 1.0000 - resnet50_conv2_block2_2_bn: 1.0000 - resnet50_conv2_block2_2_relu: 0.4846 - resnet50_conv2_blo

{'resnet50_input_1': 0.9591013590494791,
 'resnet50_conv1_pad': 0.6801383598799631,
 'resnet50_conv1_conv': 0.989959716796875,
 'resnet50_conv1_bn': 0.989959716796875,
 'resnet50_conv1_relu': 0.47998666763305664,
 'resnet50_pool1_pad': 0.3792487250434028,
 'resnet50_pool1_pool': 0.6476974487304688,
 'resnet50_conv2_block1_1_conv': 0.998046875,
 'resnet50_conv2_block1_1_bn': 0.998046875,
 'resnet50_conv2_block1_1_relu': 0.4412345886230469,
 'resnet50_conv2_block1_2_conv': 1.0,
 'resnet50_conv2_block1_2_bn': 1.0,
 'resnet50_conv2_block1_2_relu': 0.4906120300292969,
 'resnet50_conv2_block1_0_conv': 0.998046875,
 'resnet50_conv2_block1_3_conv': 1.0,
 'resnet50_conv2_block1_0_bn': 0.998046875,
 'resnet50_conv2_block1_3_bn': 1.0,
 'resnet50_conv2_block1_add': 1.0,
 'resnet50_conv2_block1_out': 0.4506549835205078,
 'resnet50_conv2_block2_1_conv': 1.0,
 'resnet50_conv2_block2_1_bn': 1.0,
 'resnet50_conv2_block2_1_relu': 0.4670085906982422,
 'resnet50_conv2_block2_2_conv': 1.0,
 'resnet50_conv2

In [12]:
# to count the number of tasks required:

def layer_count(model):
    count = 0
    for l in model.layers:
        if isinstance(l, (tf.keras.Model,models.Sequential)):
            for ll in l.layers:
                count = count + 1
        else:
            count = count + 1
    return (f'tasks_number = {count+1}')
        

In [17]:
layer_count(model_given)

'tasks_number = 176'